In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf  #calculation
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime #goes along with yfinance

## Group Assignment
### Team Number: 10
### Team Member Names: Saad Shahzad, Jeff Ni, Lucy Wu
### Team Strategy Chosen: SAFE

In [2]:
# Read csv file and store all the stock tickers in a list
header_list = ["Name"]
tickers = pd.read_csv('Tickers.csv', names = header_list)

# Set Constant
currency = 'USD'

# Set filter dates
start_date = '2021-07-02'
end_date = '2021-10-22'

# Initialize a list
filtered_list = []
deviation = []

# Make a for loop for assessing each element 
for i in range ((len(tickers))):
    current_ticker = yf.Ticker(tickers['Name'].iloc[i])
    
    # Check if stock is traded in USD
    if 'financialCurrency' in current_ticker.info and current_ticker.info['financialCurrency'] == currency:
        hist = current_ticker.history(start=start_date, end=end_date)
        volume = []
        volume = hist['Volume']
        
        # Filter out stocks that have less than 10 000 daily volume
        if volume.mean() >= 10000:
            
            # Adds ticker name to ticker list and adds standard deviation to standard deviation list
            prices = pd.DataFrame(hist['Close'])
            daily_return = prices.pct_change()
            deviation.append(daily_return.std())
            filtered_list.append(tickers['Name'].iloc[i])

            
filtered = pd.DataFrame(deviation)            
print(filtered_list)
filtered.head()

['AAPL', 'ABBV', 'LOW', 'HOOD', 'AMZN', 'AXP', 'BAC', 'BMBL', 'BK', 'SQ', 'VZ', 'CMCSA', 'SHOP', 'SHOP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'IBM', 'ORCL', 'OXY', 'DUOL', 'PEP', 'BB.TO', 'SLB', 'SO', 'SPG', 'PYPL']


,Close
0,0.012743
1,0.012421
2,0.017135
3,0.090978
4,0.015132


### Evaluating Risk
We are allowed to have a minimum of 10 stocks and a maximum of 20 stocks in our portfolio. To minimize non-systematic risk, we decided to put as many stocks as we are allowed in the portfolio. Having more stocks means greater diversity. This means more industries would be represented than if we had fewer stocks, thus reducing risk associated with specific industries. Additionally, having more stocks means that each stock makes up a smaller percent of the portfolio so drastic changes in an individual stock would not affect the entire portfolio much.  

 

If the stock list we are given has less than 20 stocks, we will be putting every stock in the list, so we have as much variation as possible. If the stock list has more than 20 stocks, we need a way to pick which stocks to put in our portfolio. We can accomplish this by filtering the stock list. First, we decided that adding risky stocks to a portfolio makes the portfolio riskier than adding only safe stocks so to ensure that we are only working with safe stocks, we took the 30 safest stocks from the list (if the list has less than 30 stocks we take as many as we can). We did not take the top 20 stocks because we wanted some extra stocks so we can take other factors of risk into account, such as correlation. Taking factors such as correlation into account reduces the odds of picking stocks that are mostly in the same industry. Standard deviation can be thought of as the average variation from the mean, so we decided that the standard deviation of percent change would be a good indicator of how much a stock fluctuates. After calculating the standard deviation of every stock, we took the 30 stocks with the lowest standard deviation to move onto the next step. 

In [3]:
# Created a dataframe with Tickers as its index and standard deviation of the Closing price.
filtered = pd.DataFrame(deviation)
filtered.rename(columns={'Close': 'Risk'}, inplace=True)
filtered['Tickers'] = filtered_list
filtered.set_index('Tickers', inplace=True)
filtered.head()

,Risk
Tickers,
AAPL,0.012743
ABBV,0.012421
LOW,0.017135
HOOD,0.090978
AMZN,0.015132


In [4]:
# Gets the 30 least risky stocks
stock_count = min(30,(len(filtered)))

#arrange in ascending oder, i.e. from least riskiest to more risky stocks. 
filtered = filtered.sort_values(by = ['Risk'])
filtered = filtered[0: stock_count]
filtered.head()

,Risk
Tickers,
VZ,0.006381
PEP,0.007793
SO,0.007807
COST,0.009899
CSCO,0.010052


In [5]:
# Turn tickers into a column and get a numbered index
filtered_v2 = filtered
filtered_v2.reset_index(inplace=True)
filtered_v2.head()

,Tickers,Risk
0,VZ,0.006381
1,PEP,0.007793
2,SO,0.007807
3,COST,0.009899
4,CSCO,0.010052


In [6]:
# Create a new dataframe with date as the index
test = yf.Ticker( filtered_v2['Tickers'].iloc[1])
hist_test = test.history(start=start_date, end=end_date)
hist_test.index
return_df = pd.DataFrame(hist_test.index)
return_df.set_index('Date', inplace=True)

In [7]:
# Creates a new dataframe to store percent change of every day of every stock
return_df = pd.DataFrame(hist_test.index)

# Creates
for i in range ((len(filtered_v2))):
    # Gets current ticker
    ticker = filtered_v2['Tickers'].iloc[i]
    current_ticker = yf.Ticker(ticker)
    
    # Gets history and prices of current ticker
    hist = current_ticker.history(start=start_date, end=end_date)
    volume = []
    volume = hist['Volume']
    prices = pd.DataFrame(hist['Close'])
    
     # Adds column for current ticker to dataframe
    return_df.loc[:, ticker] = pd.Series(list(prices.pct_change()['Close']))
    
# Set index to date and show dataframe    
return_df.set_index('Date', inplace=True)

return_df.head()

,VZ,PEP,SO,COST,CSCO,CVS,GOOG,ABBV,AAPL,ORCL,...,PYPL,SHOP,GM,SLB,SQ,OXY,BB.TO,BMBL,DUOL,HOOD
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-02,0.002665,0.004791,0.000813,0.011178,0.008856,0.002554,0.018600,0.007964,0.019596,0.028665,...,0.004291,0.004630,-0.002538,-0.008467,-0.003184,-0.008521,0.018206,-0.028241,0.001378,0.009477
2021-07-06,-0.000531,0.001477,0.001788,-0.000201,-0.010459,-0.015164,0.008173,0.004862,0.014718,0.015400,...,0.008269,0.050029,-0.025441,-0.049405,0.023312,-0.063536,-0.002649,0.000903,0.043216,0.071977
2021-07-07,0.002127,0.004426,0.000811,0.014592,0.005285,0.001971,0.002362,0.008814,0.017955,0.036230,...,0.015343,-0.011580,-0.015141,-0.015720,-0.008350,-0.033759,-0.047145,-0.067448,0.025312,0.242038
2021-07-08,-0.003971,0.000467,-0.002269,0.006104,0.000000,-0.013769,-0.006923,-0.003426,-0.009200,-0.005808,...,-0.007000,-0.037416,-0.009366,-0.004889,-0.035726,0.006106,0.010453,-0.027461,-0.027399,0.504060


### Diversifying Using Correlation
After we narrowed our tickers down to 30, we decided to pick the top 20 stocks by examining their correlations. As we learned from class, the lower the correlation between our stocks, the more diversified our portfolio would be. To reduce the risk of our portfolio, we needed to select a group of stocks that would give us the lowest correlations.  

To achieve our goal, we considered two options. The first one is to start with finding the stock that is least correlated to the first stock on our list and picking the next candidate by comparing the portfolio risk of the first two stocks with the remaining stocks and so on. The second approach, on the other hand, focuses on the correlations of pairs of stocks. To be specific, we would select 10 pairs of stocks, each having the minimum correlation to its pairing. The ultimate goal would be reducing the risk of our overall portfolio by having pairs of stocks that hedge the risk of each other. We decided to go with the second approach, as it is more unique and efficient.  

As for the weighting of the stocks, each one of them would have the same weighting as its pairing, so that we can maximize the effect of hedging.  

In [8]:
# This function pairs stocks with another stock with the lowest correlation. 
def min_correlation(ticker1, ticker_list):
    ticker2 = ""
    min_cor = 5
    

    # Loops through return_df's columns starting at the second column
    for columns in return_df.columns[1:]:
        compare = return_df[[ticker1, columns]]
        cur_correlation = compare.corr().iloc[1][0]
    
        # If the correlation calculated above is less than min_cor, the it replaces min_cor
        #   and changes ticker2 to columns to preserve the ticker name
        if cur_correlation < min_cor:
            ticker2 = columns
            min_cor = cur_correlation

    # Removes the column associated with each ticker in the pair from return_df to avoid double counting
    return_df.drop(ticker1, 1, inplace=True)
    return_df.drop(ticker2, 1, inplace=True)
    
    # Adds the 2 stocks to ticker_list and returns it
    ticker_list.append(ticker1)
    ticker_list.append(ticker2)

    return ticker_list

In [9]:
tickers_list = []

# Records the current minimal risk stock and it's lowest correlation stock
#   Does this for 10 stocks if there are 20 or more stocks, otherwise does it as much as possible 
for i in range (min(int(filtered.shape[0]/2), 10)):
    tickers_list = min_correlation(return_df.columns[0], tickers_list)
    i += 1
    
tickers_list

['VZ',
 'HOOD',
 'PEP',
 'DUOL',
 'SO',
 'OXY',
 'COST',
 'SLB',
 'CSCO',
 'BB.TO',
 'CVS',
 'SHOP',
 'GOOG',
 'IBM',
 'ABBV',
 'BMBL',
 'AAPL',
 'LOW',
 'ORCL',
 'SQ']

### Minimizing Risk With Weightings
We have used a different mechanism for weighting our stocks than the usual methods of weighting which is Price Weighting or Market Capitalization weighting. The main reason is that we are creating the portfolio for a short period.   

For our method, what we did is very similar to Equal Weighting but much more specialized. Since each stock pair must have the same weighting, we created a $\$10000$ portfolio with $\$5000$ spent on each stock in the pair. We then calculated the risk of that portfolio. We then calculated the initial weighting by dividing that number by the sum of the risk of all pair portfolios. Then we reversed the weighting so lower risk means higher weight. To acquire the weighting, we calculated the risk of every stock pair and distributed the weight more towards safer pairs and reduced the weight of riskier pairs. This method has enabled us to fairly distribute the weight based on how stable each stock pair is, while not being too biased towards any stock.  This method has helped us minimize the two significant loss factors: a sudden drop in the value of a single stock would not affect our entire portfolio value and the risky stocks would have a comparatively lower weighting than the average, which will shrink the possibility of the portfolio value going down. 

In [10]:
risk_list = []
pair_list = []
starting_amt = 10000
i = 0

# Loops through every ticker
while i < len(tickers_list) - 1:
    stocks = []
    stock_lst = tickers_list[i:i+2]
    
    # Makes list with closing prices of all stocks
    for index in stock_lst:
        stock = yf.Ticker(index)
        stock_hist = stock.history(start=start_date, end=end_date, interval='1d')
        stocks.append(stock_hist.Close)
    
    # Makes closing prices into dataframe and removes NaN values
    portfolio = pd.concat(stocks, axis=1)
    portfolio.columns = stock_lst
    portfolio = portfolio[portfolio[stock_lst[0]] == portfolio[stock_lst[0]]]

    share_count = []

    # Calculates the amount of shares purchasable for each stock
    for column in portfolio:
        share_count.append(starting_amt/(len(stock_lst)*portfolio[column].iloc[0]))
       
    portfolio_value = [0]

    # Calculates portfolio value
    for index in range(len(stock_lst)):
        portfolio_value = portfolio_value + (share_count[index] * stocks[index])

    # Calculates the risk of the portfolio and adds it to risk_list
    #pct_returns = (portfolio_value - starting_amt)*100/starting_amt
    pct_returns = portfolio_value.pct_change()
    risk_list.append(pct_returns.std())
    pair_list.append(stock_lst)
    
    # Increments by 2 to avoid counting the same stock twice
    i = i + 2

risk_list

[nan,
 nan,
 0.014119424587228055,
 0.011270835170163059,
 nan,
 0.011548779456591817,
 0.01011888272105369,
 0.01845779272891273,
 0.010891406565385507,
 0.015140998328377217]

In [11]:
# This cell reverses the weightings so lower risk means more weight
#   Ratios are preserved in the new weightings
total = sum(risk_list)

# Turns each risk into a percentage
for i in range(len(risk_list)):
    risk_list[i] = risk_list[i]/total
    
#print(risk_list)

x_list = []

# Gets each risk as a multiple of the first risk
for i in range (len(risk_list)):
    x_list.append(risk_list[0]/risk_list[i])

summing = sum(x_list)

x = 100 / summing

# Calculates the reversed weighting
weighting = []
for i in range (len(risk_list)):
    weighting.append(x_list[i] * x)
weighting

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

In [12]:
# this function makes it so that all weightings are below a given maximum and above a given minimum
def create_weight(risk_list, minimum, maximum):
    weight_list = []
    
    # The amount that will be added to non-maxed weightings
    add_amt = 0
    
    # The amount that will be reduced from non-minimized weightings
    red_amt = 0
    
    
    # Loops through all the weightings
    for i in range(len(risk_list)):
        weight = risk_list[i]
        
        # If the weighting is above max, sets it to max and stores the difference in add_amt
        if weight < minimum:
            red_amt = red_amt + minimum - weight
            weight = minimum
            
        # If the weighting is below min, sets it to min and stores the difference in red_amt
        elif weight > maximum:
            add_amt = add_amt + weight - maximum
            weight = maximum
            
        # Appends the weight to weight_list
        weight_list.append(weight)
    
        
    # If the amount to be added is more than the amount to be reduced,
    #   subtract red_amt from add_amt to get total amount to be added.
    #     Distribute that amount evenly to non-maxed weightings
    if add_amt > red_amt: 
        add_amt = add_amt - red_amt
        max_count = weight_list.count(maximum)
        add_amt = add_amt/(len(weight_list)-max_count)
        for i in range(len(weight_list)):
            if not weight_list[i] == maximum:
                weight_list[i] = weight_list[i] + add_amt
        
        
    # If the amount to be reduced is more than the amount to be added,
    #   subtract add_amt from red_amt to get total amount to be reduced.
    #     Distribute that amount evenly to non-minimized weightings
    elif add_amt < red_amt:
        red_amt = red_amt - add_amt
        min_count = weight_list.count(minimum)
        red_amt = red_amt/(len(weight_list)-min_count)
        for i in range(len(weight_list)):
            if not weight_list[i] == minimum:
                weight_list[i] = weight_list[i] - red_amt
    
    # Check if the new weighting is valid. If not, run this function on the new weighting
    if not valid_weighting(weight_list, minimum, maximum):
        weight_list = create_weight(weight_list, minimum, maximum)
        
    return weight_list
        
    

In [13]:
# Check if a weighting is valid
def valid_weighting(weighting_list, mini, maxi):
    
    # Check if all values are below maxi and above mini
    for i in range(len(weighting_list)):
        if weighting_list[i] < mini or weighting_list[i] > maxi:
            return False
        
    return True

In [14]:
# Gets the weighting of each pair of stocks if they were all weighted equally
avg_weight = 100 / len(weighting)

# Creates the weighting for each pair of stocks
temp_weight = create_weight(weighting, avg_weight - 3, avg_weight + 3)
weight_list = []

# Converts the pair weighting into individual stock weightings
for index in range(len(temp_weight)):
    weight_list.append(temp_weight[index]/2)
    weight_list.append(temp_weight[index]/2)

weight_list

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [15]:
# Making the final portfolio
starting_amt = 100000
prices = []
shares = []
value = []

# End date is non-inclusive, we set start date a bit earlier than Nov 26
#  So if there is no data on Nov 26, we get the latest data before Nov 26
start_date = '2021-11-20'
end_date = '2021-11-27'

# Gets a list of prices
for ticker in tickers_list:
    cur_ticker = yf.Ticker(ticker)
    hist = cur_ticker.history(start=start_date, end=end_date, interval='1d')
    prices.append(hist['Close'].iloc[len(hist['Close'])-1])
    
# Creates a list of values and shares    
for i in range(len(tickers_list)):
    value.append((weight_list[i]/100)*starting_amt)
    shares.append(value[i]/prices[i])
    
# Puts all the information in a dataframe
FinalPortfolio = pd.DataFrame(tickers_list)
FinalPortfolio.rename(columns={0:'Ticker'}, inplace=True)
FinalPortfolio['Price'] = prices
FinalPortfolio['Shares'] = shares
FinalPortfolio['Value'] = value
FinalPortfolio['Weighting'] = weight_list
FinalPortfolio.index += 1

FinalPortfolio.head()

,Ticker,Price,Shares,Value,Weighting
1,VZ,51.799999,NaN,NaN,NaN
2,HOOD,27.920000,NaN,NaN,NaN
3,PEP,161.139999,NaN,NaN,NaN
4,DUOL,122.580002,NaN,NaN,NaN
5,SO,62.040001,NaN,NaN,NaN


In [16]:
# Output to CSV
Stocks = pd.DataFrame(FinalPortfolio['Ticker'])
Stocks['Shares'] = FinalPortfolio['Shares']
Stocks.to_csv('Stocks_Group_10.csv')
Stocks

,Ticker,Shares
1,VZ,NaN
2,HOOD,NaN
3,PEP,NaN
4,DUOL,NaN
5,SO,NaN
6,OXY,NaN
7,COST,NaN
8,SLB,NaN
9,CSCO,NaN
10,BB.TO,NaN


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Saad Shahzad, Lucy Wu, Jeff Ni